In [1]:
import torch
import torch_geometric

import math

import torch

from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
import torch.nn.functional as F

from GCNlayers import GraphConvolution
from GCNmodel import GCN

In [3]:
# 创造初始边界条件

numNodes = 100 #节点个数
numEdges = 2000 #边的个数

# 设置随机种子
torch.manual_seed(0)

row = torch.randint(0,numNodes,(numEdges,))
col = torch.randint(0,numNodes,(numEdges,))

edgeIndex = torch.stack([row, col], dim=0) #按照列合并 [2,numEdges]
print(edgeIndex.shape)

edgeIndex = torch.cat([edgeIndex, edgeIndex.flip(0)], dim=1) # 按照每行翻转后按行cat
print(edgeIndex.shape)



# 去掉重复的
edgeIndex = edgeIndex.unique(dim=1)

# 利用pyg生成网络
data = torch_geometric.data.Data(edge_index=edgeIndex)

adj = torch_geometric.utils.to_dense_adj(data.edge_index)
print(adj.shape) # [1,numNodes,numNodes]

# 构建随机节点特征模型
nodeFeatures = 16
nodeTarget = torch.randn((numNodes,nodeFeatures))


torch.Size([2, 2000])
torch.Size([2, 4000])
torch.Size([1, 100, 100])


In [4]:

# 利用pyg做 trainMask 和 testMask
data.train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)

data.train_mask[:int(0.8 * numNodes)] = 1
data.test_mask[int(0.8 * numNodes):] = 1

d:\anaconda\envs\pytorchenv\Lib\site-packages\torch_geometric\data\storage.py:452: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_index'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(
d:\anaconda\envs\pytorchenv\Lib\site-packages\torch_geometric\data\storage.py:452: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_index', 'train_mask'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


In [15]:
# 读取AML数据
import pandas as pd
data = pd.read_csv('D:\\data\\HI-Small_Trans.csv')

In [39]:
data.columns

Index(['Timestamp', 'From Bank', 'Account', 'To Bank', 'Account.1',
       'Amount Received', 'Receiving Currency', 'Amount Paid',
       'Payment Currency', 'Payment Format', 'Is Laundering', 'NodeID1',
       'NodeID2'],
      dtype='object')

In [25]:
# 查看Laundering的分布
data['Is Laundering'].value_counts()

Is Laundering
0    5073168
1       5177
Name: count, dtype: int64

In [26]:
# 按照时间戳划分训练集和测试集
data['Timestamp'] = pd.to_datetime(data['Timestamp'])

In [ ]:
# 将AML数据转换为PyG格式

# 创建unique节点ID
# 采用将bank + account的方式

def createNodeId(data):
    data['NodeID1'] = data['From Bank'].astype(str) + '_' + data['Account'].astype(str)
    data['NodeID2'] = data['To Bank'].astype(str) + '_' + data['Account.1'].astype(str)
    return data

data = createNodeId(data)

from sklearn.preprocessing import LabelEncoder

allNodes = pd.concat([data['NodeID1'], data['NodeID2']],axis = 0)
encoder = LabelEncoder()
encoder.fit(allNodes)

data['NodeID1'] = encoder.transform(data['NodeID1'])
data['NodeID2'] = encoder.transform(data['NodeID2'])


allPayment = pd.concat(
    [data['Receiving Currency'],data['Payment Currency']],axis = 0
)

encoder = LabelEncoder()
encoder.fit(allPayment)
data['Receiving Currency'] = encoder.transform(data['Receiving Currency'])
data['Payment Currency'] = encoder.transform(data['Payment Currency'])

paymentFormat = data['Payment Format']
encoder = LabelEncoder()
encoder.fit(paymentFormat)
data['Payment Format'] = encoder.transform(data['Payment Format'])


In [41]:
q80data = data['Timestamp'].quantile(0.8)
dataTrain = data[data['Timestamp'] < q80data]
dataTest = data[data['Timestamp'] >= q80data]

In [42]:
dataTrain.columns

Index(['Timestamp', 'From Bank', 'Account', 'To Bank', 'Account.1',
       'Amount Received', 'Receiving Currency', 'Amount Paid',
       'Payment Currency', 'Payment Format', 'Is Laundering', 'NodeID1',
       'NodeID2'],
      dtype='object')

In [43]:
dataTrain = dataTrain[['Timestamp','NodeID1','NodeID2',
'Amount Received','Receiving Currency','Amount Paid','Payment Currency','Payment Format']]
dataTest = dataTest[['Timestamp','NodeID1','NodeID2',
'Amount Received','Receiving Currency','Amount Paid','Payment Currency','Payment Format']]

In [49]:
row = data['NodeID1']
col = data['NodeID2']

row = torch.tensor(row.values)
col = torch.tensor(col.values)

edgeIndex = torch.stack([row, col], dim=0)


In [ ]:
def csvToPyg(data):

    row = data['NodeID1']
    col = data['NodeID2']

    row = torch.tensor(row.values)
    col = torch.tensor(col.values)

    edgeIndex = torch.stack([row, col], dim=0)
    

    return Data